In [14]:
#Initialize a list of all NBA teams
nba_teams = [
    'ATL',  # Atlanta Hawks
    'BOS',  # Boston Celtics
    'BKN',  # Brooklyn Nets
    'CHA',  # Charlotte Hornets
    'CHI',  # Chicago Bulls
    'CLE',  # Cleveland Cavaliers
    'DAL',  # Dallas Mavericks
    'DEN',  # Denver Nuggets
    'DET',  # Detroit Pistons
    'GSW',  # Golden State Warriors
    'HOU',  # Houston Rockets
    'IND',  # Indiana Pacers
    'LAC',  # LA Clippers
    'LAL',  # Los Angeles Lakers
    'MEM',  # Memphis Grizzlies
    'MIA',  # Miami Heat
    'MIL',  # Milwaukee Bucks
    'MIN',  # Minnesota Timberwolves
    'NOP',  # New Orleans Pelicans
    'NYK',  # New York Knicks
    'OKC',  # Oklahoma City Thunder
    'ORL',  # Orlando Magic
    'PHI',  # Philadelphia 76ers
    'PHX',  # Phoenix Suns
    'POR',  # Portland Trail Blazers
    'SAC',  # Sacramento Kings
    'SAS',  # San Antonio Spurs
    'TOR',  # Toronto Raptors
    'UTA',  # Utah Jazz
    'WAS'   # Washington Wizards
]

In [15]:
#Define a url with a placeholder 
team_url = "https://www.basketball-reference.com/teams/{}/2024.html"

In [16]:
import requests

#Iterate through teams and retrieve their data
for team in nba_teams:
    url = team_url.format(team)
    data = requests.get(url)

    #Save the data in the team_data folder as an html file
    with open("team_data/{}.html".format(team), "w+") as f:
        f.write(data.text)

In [17]:
from bs4 import BeautifulSoup

In [35]:
team = "TOR"

#Open and read html file from the file path
with open(f"team_data/{team}.html") as f:
    page = f.read()

In [19]:
#Create a beautiful soup html parser object
soup = BeautifulSoup(page, "html.parser")

In [48]:
# Find the table with id "roster"
roster_table = soup.find('table', {'id': 'roster'})

In [49]:
# Find all elements with the data-stat="player" attribute
roster = roster_table.find_all('td', {'data-stat': 'player'})

In [38]:
import os 

In [50]:
def create_folder(folder_name):
    try:

        # Check if the folder already exists
        if not os.path.exists(folder_name):
            
            # Create the folder
            os.makedirs(folder_name)
            
    except Exception as e:
        print(f"Error: {e}")

In [51]:
for player in roster:
    # Find the <a> tag within the <td> element
    link = player.find('a')
    
    # Check if an <a> tag is found
    if link:
        # Get the href attribute value
        url = link['href']
        
        # Load the content of the player's page using requests
        response = requests.get(f"https://www.basketball-reference.com/{url}") 

        #Create a new folder to holder team data if it does not already exist
        folder_name = f"{team}_player_data"
        create_folder(folder_name)

        # Get the text content (name of the player)
        player_name = link.text.strip()

        #Save the data in the team_data folder as an html file
        with open(folder_name + "/{}.html".format(player_name), "w+") as f:
            f.write(data.text)

In [52]:
import pandas as pd